<a href="https://colab.research.google.com/github/harshsinha-12/LangChain/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Environment Variable with the Key

In [7]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-d'

# Installing important required Libraries

In [33]:
pip install openai

# FrameWork

In [34]:
pip install langchain

In [35]:
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0)

# Creative from scale 0 to 1, 0 safer, 1 is risky but creative......0.6-0.7 good

name = llm("I want to open a resturant for Italian food. Suggest a fancy name for it.") #Input Question like for the LLM
print(name)



La Dolce Vita Ristorante


# Creating a Prompt Template with an Input variable {Cuisine}

In [37]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for it."
)
prompt_template_name.format(cuisine="Italian")

'I want to open a restaurant for Italian food. Suggest a fancy name for it.'

# Chain Object where (llm, prompt template) is used just to use the {Cuisine} : Simple Chain : One Chain with cuisine as input and the name as output

In [38]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("American")

'\n\nThe Gilded Grill'

# Simple Sequential Chain like a second chain.....Cuisine-----> Resturant Name ------> Menu of Resturant (just one output)

In [39]:
llm = OpenAI(temperature = 0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for it."
)
name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template = "Suggest some menu items for {resturant_name}. Return it as a comma seperator."
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items)


In [40]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Indian Mughlai")
print(response)



Tandoori Chicken, Butter Chicken, Chicken Tikka Masala, Lamb Rogan Josh, Saag Paneer, Aloo Gobi, Vegetable Biryani, Naan, Samosas, Gulab Jamun


# Simple Sequential Chain like a second chain.....Cuisine-----> Resturant Name ------> Menu of Resturant (Multiple Output)

In [47]:
llm = OpenAI(temperature = 0.7)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for it."
)
name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = "resturant_name")

llm = OpenAI(temperature = 0.7)

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template = "Suggest some menu items for {resturant_name}."
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items, output_key = 'menu_items')


In [2]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['resturant_name', 'menu_items']
)
chain({'cuisine' : "Indian Mughlai"})


ModuleNotFoundError: ignored